In [0]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.window import Window

In [0]:
df = spark.read.format('csv').option('inferSchema', True).option('header', True).load('/FileStore/tables/BigMart_Sales.csv')

In [0]:
df.display()

In [0]:
df.limit(10).display()

In [0]:
df_lim = df.limit(5)

In [0]:
df_lim.display()

In [0]:
df.groupBy('Item_Type').agg(sum('Item_MRP').alias('Total_MRP')).display()

In [0]:
df.groupBy('Item_Type').pivot('Outlet_Size').agg(sum('Item_MRP'),avg('Item_MRP')).display()

In [0]:
df.groupBy('Item_Type').pivot('Outlet_Size').agg(sum('Item_MRP'),avg('Item_MRP'), count('Item_MRP')).display()

In [0]:
df.withColumn('veg_flag', when(lower(col('Item_Type')) == 'meat', 'Non_veg').otherwise('Veg')).display()

In [0]:
df_veg = df.withColumn('veg_flag', when(lower(col('Item_Type')) == 'meat', 'Non_veg').otherwise('Veg'))

In [0]:
df_veg.limit(5).display()

In [0]:
df_veg.withColumn('flag_expensive', when((col('veg_flag') == 'Veg') & (col('Item_MRP') <= 100 ), 'veg_inexpensive' )\
                                    .when((col('veg_flag') == 'Veg') & (col('Item_MRP') > 100), 'veg_expensive')\
                                        .otherwise('non_veg')  ).display()

In [0]:
dataj1 = [('1','gaur','d01'),
          ('2','kit','d02'),
          ('3','sam','d03'),
          ('4','tim','d03'),
          ('5','aman','d05'),
          ('6','nad','d06')] 

schemaj1 = 'emp_id STRING, emp_name STRING, dept_id STRING' 

df1 = spark.createDataFrame(dataj1,schemaj1)

dataj2 = [('d01','HR'),
          ('d02','Marketing'),
          ('d03','Accounts'),
          ('d04','IT'),
          ('d05','Finance')]

schemaj2 = 'dept_id STRING, department STRING'

df2 = spark.createDataFrame(dataj2,schemaj2)

In [0]:
df1.display()
df2.display()

In [0]:
df1.join(df2, df1['dept_id'] == df2['dept_id'], 'inner' ).display()

In [0]:
df1.join(df2, df1['dept_id'] == df2['dept_id'], 'left').display()

In [0]:
df1.join(df2, df1['dept_id'] == df2['dept_id'], 'right').display()

In [0]:
df1.join(df2, df1['dept_id'] == df2['dept_id'], 'anti').display()

In [0]:
df.display()

In [0]:
df.withColumn('row_number', row_number().over(Window.orderBy('Item_Identifier')) ).display()

In [0]:
df.withColumn('row_number', row_number().over(Window.orderBy('Item_Identifier')) )\
    .withColumn('rank', rank().over(Window.orderBy('Item_Identifier')))\
        .withColumn('dens_rank', dense_rank().over(Window.orderBy('Item_Identifier'))).display()

In [0]:
df.withColumn('cum_sum', sum('Item_MRP').over(Window.orderBy('Item_Type').rowsBetween(Window.unboundedPreceding, Window.currentRow) )  ).display()

In [0]:
df.withColumn('cum_sum', sum('Item_MRP').over(Window.orderBy('Item_Type').rowsBetween(Window.unboundedPreceding, Window.currentRow) )  )\
    .select(['Item_Type','Item_MRP','cum_sum' ]).display()

In [0]:
def temp_func(x):
    return x * x

In [0]:
my_udf = udf(temp_func)

In [0]:
df.withColumn('my_new_func', my_udf('Item_MRP')).display()

In [0]:
df_my_func = df.withColumn('my_new_func', my_udf('Item_MRP'))

In [0]:
df_my_func.limit(5).display()

In [0]:
dbutils.fs.ls('/FileStore/tables')

In [0]:
df_my_func.write.format('csv')\
    .mode('append')\
        .option('header', True)\
        .save('/FileStore/tables/my_func_data.csv')

In [0]:
dbutils.fs.ls('/FileStore/tables')

In [0]:
new_df = spark.read.format('csv').option('inferschema', True).option('header', True).load('dbfs:/FileStore/tables/my_func_data.csv/')

new_df.display()

In [0]:
new_df_tmp = spark.read.format('csv').option('inferschema', True).load('dbfs:/FileStore/tables/my_func_data.csv')

new_df_tmp.display()

In [0]:
df_my_func.write.format('csv')\
    .mode('append')\
        .option('header', True)\
        .save('/FileStore/tables/my_func_data.csv')

In [0]:
dbutils.fs.ls('/FileStore/tables')

In [0]:
new_df = spark.read.format('csv').option('inferschema', True).option('header', True).load('dbfs:/FileStore/tables/my_func_data.csv/')

new_df.display()

In [0]:
df_my_func.display()

In [0]:
df_my_func.write.format('csv')\
    .mode('overwrite')\
        .option('header', True)\
        .save('/FileStore/tables/my_func_data.csv')

In [0]:
new_df = spark.read.format('csv').option('inferschema', True).option('header', True).load('dbfs:/FileStore/tables/my_func_data.csv/')

new_df.display()

In [0]:
new_df.write.format('csv').mode('error').option('header', True).save('/FileStore/tables/my_func_data.csv')

In [0]:
new_df.write.format('csv')\
        .mode('ignore').option('header', True)\
            .save('/FileStore/tables/my_func_data.csv')

In [0]:
new_df.write.format('parquet')\
        .mode('overwrite')\
            .save('/FileStore/tables/my_func_data.csv')

In [0]:
dbutils.fs.ls('/FileStore/tables/')

In [0]:
new_df = spark.read.format('parquet').option('inferschema', True).option('header', True).load('dbfs:/FileStore/tables/my_func_data.csv/')

new_df.display()

In [0]:
new_df_1 = spark.read.format('csv').option('inferschema', True).option('header', True).load('dbfs:/FileStore/tables/my_func_data.csv/')

new_df_1.display()

In [0]:
df.write.format('parquet')\
    .mode('overwrite')\
        .saveAsTable('my_table_v1')

In [0]:
df.createTempView('my_tmp_view')

In [0]:
%sql
SELECT * FROM my_tmp_view WHERE Item_Fat_Content = 'Regular'

In [0]:
df_sql = spark.sql("SELECT * FROM my_tmp_view WHERE Item_Fat_Content = 'Regular' ")

In [0]:
df_sql.display()